<a href="https://colab.research.google.com/github/shrashti-19/Skin_disease/blob/main/VisionNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install kaggle

In [4]:
from google.colab import files
files.upload()


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shrashtidwivedi","key":"35416131a488f9c4bc7947c818ff2dce"}'}

In [6]:
!ls

'kaggle (1).json'   sample_data


In [7]:
!mkdir -p ~/.kaggle
!mv "kaggle (1).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
# Downloading dataset from Kaggle
!kaggle datasets download -d haroonalam16/20-skin-diseases-dataset

# Unzip it
!unzip -q 20-skin-diseases-dataset.zip -d skin_diseases


Dataset URL: https://www.kaggle.com/datasets/haroonalam16/20-skin-diseases-dataset
License(s): other


In [9]:
!pip install -q transformers


In [26]:
import os
from PIL import Image
from torch.utils.data import Dataset

class SkinDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label2id = {}
        self.id2label = {}

        class_folders = sorted([
            folder for folder in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, folder))
        ])

        for idx, label in enumerate(class_folders):
            self.label2id[label] = idx
            self.id2label[idx] = label
            folder_path = os.path.join(root_dir, label)
            for fname in os.listdir(folder_path):
                img_path = os.path.join(folder_path, fname)
                if os.path.isfile(img_path) and not fname.startswith('.'):
                    self.image_paths.append(img_path)
                    self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [15]:
!ls skin_diseases

Dataset


In [27]:
train_path = "skin_diseases/Dataset/train"
test_path = "skin_diseases/Dataset/test"

train_dataset = SkinDiseaseDataset(train_path, transform=transform)
test_dataset = SkinDiseaseDataset(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [25]:
!ls skin_diseases/Dataset/test/

'Acne and Rosacea Photos'
'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions'
'Atopic Dermatitis Photos'
'Bullous Disease Photos'
'Cellulitis Impetigo and other Bacterial Infections'
'Eczema Photos'
'Exanthems and Drug Eruptions'
'Herpes HPV and other STDs Photos'
'Light Diseases and Disorders of Pigmentation'
'Lupus and other Connective Tissue diseases'
'Melanoma Skin Cancer Nevi and Moles'
'Poison Ivy Photos and other Contact Dermatitis'
'Psoriasis pictures Lichen Planus and related diseases'
'Seborrheic Keratoses and other Benign Tumors'
'Systemic Disease'
'Tinea Ringworm Candidiasis and other Fungal Infections'
'Urticaria Hives'
'Vascular Tumors'
'Vasculitis Photos'
'Warts Molluscum and other Viral Infections'


In [28]:
from transformers import ViTForImageClassification

num_classes = 20  # You have 20 skin disease categories

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_classes,
    ignore_mismatched_sizes=True  # Important to avoid classifier shape mismatch
)




Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([20]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([20, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import torch
from torch import nn
from torch.optim import AdamW

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)



In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop (1 epoch example)
model.train()
for images, labels in tqdm(train_loader, desc="Training"):
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    outputs = model(images).logits
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("Training completed for 1 epoch.")


Training:   1%|          | 2/164 [01:14<1:41:54, 37.74s/it]